In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from transformers.wl_graph_kernel_transformer import WLGraphKernelTransformer
import graph_helper
import dataset_helper
import wl
import os
import pickle
import json

results = {}
for dataset_name in dataset_helper.get_all_available_dataset_names():
    graph_dataset_caches = ['dataset_graph_cooccurrence_{}_{}.npy'.format(window_size, dataset_name) for window_size in range(1, 5)]
    graph_dataset_caches += ['dataset_graph_gml_{}-single.npy'.format(dataset_name)]
    for graph_dataset_cache_file in graph_dataset_caches:
        cache_file = dataset_helper.CACHE_PATH + '/' + graph_dataset_cache_file
        result_file = 'data/results/{}.results.npy'.format(graph_dataset_cache_file)
        print('{}\tDataset File: {}'.format('#' * 10, graph_dataset_cache_file))
        
        if os.path.exists(result_file):
            print('Already calculated result: {}'.format(result_file))
            continue

        if not os.path.exists(cache_file):
            print('Could not find cachefile: "{}". Skipping.'.format(cache_file))
            continue
            
        X, Y = dataset_helper.get_dataset(dataset_name, use_cached = True, cache_file = cache_file)
        X, Y = np.array(X), np.array(Y)

        p = Pipeline([
            ('wl_transformer', WLGraphKernelTransformer()),
            ('clf', sklearn.linear_model.PassiveAggressiveClassifier())
        ])

        param_grid = dict(
            wl_transformer__H= [1],
            wl_transformer__n_jobs= [1],
            clf__n_iter=[100],
            clf__class_weight = ['balanced']
        )

        cv = GridSearchCV(estimator = p, param_grid=param_grid, cv=3, scoring = 'f1_macro', n_jobs=1, verbose = 11)
        gscv_result = cv.fit(X, Y)
        with open(result_file, 'wb') as f:
            pickle.dump(gscv_result.cv_results_, f)
        results[graph_dataset_cache_file] = gscv_result.cv_results_
        #gscv_result.best_estimator_, gscv_result.cv_results_

##########	Dataset File: dataset_graph_cooccurrence_1_ling-spam.npy
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1 
WLGraphKernelTransformer.fit: len(X)=1928, H=1
WLGraphKernelTransformer.fit: Found empty graph in training set!
WLGraphKernelTransformer.transform: len(X)=1928, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
WLGraphKernelTransformer.transform: len(X)=965, H=1
WLGraphKernelTransformer.transform: len(X)=1928, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
[CV]  wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1, score=0.648080, total= 2.4min
[CV] wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1 
WLGraphKernelTransformer.fit: len(X)=1929, H=1


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


WLGraphKernelTransformer.transform: len(X)=1929, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
WLGraphKernelTransformer.transform: len(X)=964, H=1
WLGraphKernelTransformer.transform: len(X)=1929, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
[CV]  wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1, score=0.811665, total= 2.4min
[CV] wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1 
WLGraphKernelTransformer.fit: len(X)=1929, H=1


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.8min remaining:    0.0s


WLGraphKernelTransformer.transform: len(X)=1929, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
WLGraphKernelTransformer.transform: len(X)=964, H=1
WLGraphKernelTransformer.transform: len(X)=1929, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
[CV]  wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1, score=0.830984, total= 2.4min
WLGraphKernelTransformer.fit: len(X)=2893, H=1


...

/Users/davidgengenbach/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/davidgengenbach/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


WLGraphKernelTransformer.transform: len(X)=7616, H=1
WLGraphKernelTransformer.transform: using pre-calculated phi list
[CV]  wl_transformer__n_jobs=1, clf__n_iter=100, clf__class_weight=balanced, wl_transformer__H=1, score=0.058345, total= 1.7min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.4min finished


WLGraphKernelTransformer.fit: len(X)=11374, H=1


KeyboardInterrupt: 